In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Activation
from keras.layers import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, AvgPool2D, BatchNormalization, Dropout, Activation, MaxPool2D
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import to_categorical

In [ ]:
xy_savez_load = np.load('./cnn_np.npz')
np_images = xy_savez_load['x_data']
np_label = xy_savez_load['y_data']
# split data
x_train, x_val, y_train, y_val = train_test_split(np_images,np_label,test_size = 0.2, random_state = 42)
# 정규화
scaler = MinMaxScaler()
x_train_norm = x_train[:,:,:,:]
x_val_norm = x_val[:,:,:,:]
x_train_norm = x_train_norm.reshape([-1, 96*96])
x_val_norm = x_val_norm.reshape([-1,96*96])
x_train_norm = scaler.fit_transform(x_train_norm)
x_val_norm = scaler.fit_transform(x_val_norm)
# resize
x_train_norm = x_train_norm.reshape([-1,96,96,3])
x_val_norm = x_val_norm.reshape([-1,96,96,3])
# one-hot
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)


class CNNModel:
    def __init__(self, epoch, output):
        self.epoch = epoch
        self.output = output
    def build_model(self):
        output_class = 6

        self.model = Sequential()
        self.model.add(Conv2D(24, (3,3), input_shape = (96,96,3),  use_bias = False, kernel_initializer='he_normal', kernel_regularizer = regularizers.l2(0.01)))
        self.model.add(BatchNormalization(axis = -1))
        self.model.add(Activation('relu'))

        self.model.add(Conv2D(18, (2,2), use_bias = False,  kernel_initializer='he_normal', kernel_regularizer = regularizers.l2(0.01)))
        self.model.add(BatchNormalization(axis = -1))
        self.model.add(Activation('relu'))

        self.model.add(MaxPooling2D(pool_size=(2,2)))

        self.model.add(Conv2D(13, (2,2), use_bias = False,  kernel_initializer='he_normal', kernel_regularizer = regularizers.l2(0.01)))
        self.model.add(BatchNormalization(axis = -1))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Flatten())

        self.model.add(Dense(184,use_bias  = True, kernel_initializer = 'he_normal',bias_initializer = 'he_normal', kernel_regularizer = regularizers.l2(0.01)))
        self.model.add(BatchNormalization(axis = -1))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))

        self.model.add(Dense(256,use_bias  = True, kernel_initializer = 'he_normal',bias_initializer = 'he_normal', kernel_regularizer = regularizers.l2(0.01)))
        self.model.add(BatchNormalization(axis = -1))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))


        self.model.add(Dense(128,use_bias  = True, kernel_initializer = 'he_normal',bias_initializer = 'he_normal', kernel_regularizer = regularizers.l2(0.01)))
        self.model.add(BatchNormalization(axis = -1))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))

        self.model.add(Dense(48,use_bias  = True, kernel_initializer = 'he_normal',bias_initializer = 'he_normal', kernel_regularizer = regularizers.l2(0.01)))
        self.model.add(BatchNormalization(axis = -1))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))

        self.model.add(Dense(output_class,use_bias  = True, kernel_initializer = 'he_normal',bias_initializer = 'he_normal', kernel_regularizer = regularizers.l2(0.01)))
        self.model.add(BatchNormalization(axis = -1))
        self.model.add(Activation('softmax'))

        self.model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
        def training_model(self, file_name):
            csv_logger = CSVLogger(file_name + '.csv', separator=',', append=True)
            log = model.fit(x_train_norm, y_train, epochs = 30, batch_size = 10, validation_data= [x_val_norm,y_val])
            self.model.save(file_name + '.h5')
            
xy_data_path = './cnn_30.npz'

model1 = CNNmodel(30)
model1.build_model()
model1.training_model('model3_norm_7000_200', xy_savez_load)

